In [7]:
 !pip install transformers gradio sentencepiece datasets --quiet


In [2]:
import gradio as gr
from transformers import pipeline
import re

In [3]:
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


In [4]:
def extract_action_items(text):
    """
    Extracts action items based on trigger patterns.
    """
    patterns = [
        r"\bwill\b.*?(?:\.|$)",
        r"\bneed to\b.*?(?:\.|$)",
        r"\bshould\b.*?(?:\.|$)",
        r"\bassign\b.*?(?:\.|$)",
        r"\bdeadline\b.*?(?:\.|$)"
    ]
    action_items = []
    for pattern in patterns:
        matches = re.findall(pattern, text, flags=re.IGNORECASE)
        action_items.extend([m.strip() for m in matches])
    return list(set(action_items))  # remove duplicates

In [5]:
def process_meeting(meeting_text):
    # Summarize
    summary = summarizer(meeting_text, max_length=130, min_length=30, do_sample=False)[0]['summary_text']

    # Extract action items
    actions = extract_action_items(meeting_text)

    # Format output
    if actions:
        actions_str = "\n".join(f"- {a}" for a in actions)
    else:
        actions_str = "No clear action items found."

    return summary, actions_str

In [6]:
with gr.Blocks() as demo:
    gr.Markdown("# 📋 Smart Meeting Summarizer & Action Item Extractor")
    gr.Markdown("Paste meeting transcript text below and get a concise summary plus action items.")

    input_text = gr.Textbox(label="Meeting Transcript", lines=12, placeholder="Paste your meeting text here...")
    summary_output = gr.Textbox(label="Meeting Summary", lines=6)
    actions_output = gr.Textbox(label="Action Items", lines=6)

    run_btn = gr.Button("Process")
    run_btn.click(process_meeting, inputs=input_text, outputs=[summary_output, actions_output])

demo.launch(debug=True)

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://a7c149bb3854528cb8.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://a7c149bb3854528cb8.gradio.live
